In [1]:
import pandas as pd
import datetime as dt
from difflib import SequenceMatcher
import difflib
from fuzzywuzzy import fuzz
import re

/Users/Andy/.pyenv/versions/3.10.11/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df1 = pd.read_csv('postcode.csv', sep=',', encoding='latin-1')

In [3]:
df2 = pd.read_csv('test_postal.csv', sep=',', encoding='latin-1')

In [4]:
regex = re.compile('[^a-zA-Z0-9]+')
for col in df2.columns:
    # Convert the column to a string before using the str accessor might need to change back to int depend on db
    matches = df2[col].astype(str).str.contains(regex)
    if matches.any():
        rows = df2.loc[matches]
        print(f'Column "{col}" contains special characters in the following rows:')
        print(rows)

        # hmmmm cannot replace...
        # rows.replace(regex, "")
        # print(rows)


Column "postal" contains special characters in the following rows:
      postal
24   789872#
141  638377#
268  259621#
306  437560#


In [5]:
max_lengths = {'postal': 6}

for i, row in df2.iterrows():
    for col, max_length in max_lengths.items():
        if len(str(row[col])) > max_length:
            print(f"Row {i+1} @ Col {col}: {row[col]}")

Row 25 @ Col postal: 789872#
Row 142 @ Col postal: 638377#
Row 269 @ Col postal: 259621#
Row 307 @ Col postal: 437560#


In [6]:
df3 = pd.read_csv('postcodetest.csv', sep=',', encoding='latin-1')

In [7]:
merged_df = pd.merge(df1, df3, on='postal', how='inner')
print(merged_df)

       postal  latitude_x  longtitude_x  \
0      398614    1.312763    103.883519   
1      398721    1.312390    103.881504   
2      629875    1.309135    103.679463   
3      439731    1.305466    103.895674   
4      659592    1.344619    103.749789   
...       ...         ...           ...   
25261  210662    1.306246    103.849956   
25262  210663    1.305681    103.850858   
25263  210664    1.306133    103.851113   
25264  579792    1.347091    103.854878   
25265  168895    1.285903    103.834011   

                                    searchval_x blk_no_x  \
0                                      # 1 LOFT        1   
1                                    # 1 SUITES        1   
2                 1 BENOI ROAD SINGAPORE 629875        1   
3              1 BOSCOMBE ROAD SINGAPORE 439731        1   
4      1 BUKIT BATOK STREET 22 SINGAPORE 659592        1   
...                                         ...      ...   
25261             ZHUJIAO CENTRE (TEKKA MARKET)      662   
252

In [8]:
# result_df = merged_df.drop(columns=['', ''])
# print(result_df)

In [9]:
# Trying to merge while delete the col, not working yet

# merged_df = pd.merge(df1[['blk_no', 'road_name']], df2, on='postal', how='inner')
# print(merged_df)

In [10]:
today_date = dt.date.today().strftime("%Y%m%d")
print(today_date)

20230428


In [11]:
# print(df1.iloc[4, 2])

In [12]:
# SequenceMatcher(None, 'asdasd', 'dsadsa').ratio()

In [13]:
# df = pd.DataFrame({'col1': ['apple', 'banana', 'carrot', 'dog', 'elephant'], 'col2': ['apple', 'bandana123', 'care123', '123cat', 'eagl232e']})

# threshold = 70

# for i, row in df.iterrows():
#     fratio = fuzz.ratio(row['col1'], row['col2'])
#     if fratio >= threshold:
#         print(f"Columns 'col1' and 'col2' in row {i+1} have a similarity of {fratio:.2f}%.")


In [14]:
# threshold = 0.4

# for i, row in df.iterrows():
#     ratio = SequenceMatcher(None, row['col1'], row['col2']).ratio()
#     if ratio >= threshold:
#         print(f"Columns 'col1' and 'col2' in row {i+1} have a similarity of {ratio*100:.2f}%.")

In [15]:
df = pd.DataFrame({'col1': ['apple', 'banana', 'carrot', 'cat dog cat', 'elephant','testing', 'testing1234567789', 'testing testing'],
                   'col2': ['apple', 'bandana123', 'care123', 'cat dog dog', 'eagl232e', 'testloling', 'testing', 'test testing test']})

In [16]:
# def get_similar_rows_ver1(df, col1, col2, threshold):
#     similar_rows = []
#     for i, row in df.iterrows():
#         ratio = SequenceMatcher(None, row[col1], row[col2]).ratio()
#         if ratio >= threshold:
#             similar_rows.append(i)
#     return similar_rows

In [17]:
def get_similar_rows_ver2(df, col1, col2, threshold):
    for i, row in df.iterrows():
        ratio = SequenceMatcher(None, row[col1], row[col2]).ratio()
        if ratio == 1.0:
            df.loc[i, col2] = row[col2].replace(row[col1], "")
            print(f"Row {i+1}: {col2} is deleted due to {row[col1]} in {col2}.")
        elif ratio >= threshold:
            print(f"Row {i+1}: have a similarity of {ratio*100:.2f}% on {col1} & {col2}.")

In [18]:
def get_similar_rows_ver3(df, col1, col2, threshold):
    for i, row in df.iterrows():
        ratio = SequenceMatcher(None, row[col1], row[col2]).ratio()
        if ratio == 1.0:
            col1dup = row[col1]
            col2dup = row[col2]
            df.loc[i, col2] = col2dup.replace(col1dup, "")
            print(f"Row {i+1}: {col2} is deleted due to {col1dup} in {col2}.")
        elif ratio >= threshold:
            print(f"Row {i+1}: have a similarity of {ratio*100:.2f}% on {col1} & {col2}.")

In [19]:
get_similar_rows_ver3(df, "col1", "col2", 0.7)
print(df)

Row 1: col2 is deleted due to apple in col2.
Row 2: have a similarity of 75.00% on col1 & col2.
Row 4: have a similarity of 72.73% on col1 & col2.
Row 6: have a similarity of 82.35% on col1 & col2.
Row 8: have a similarity of 75.00% on col1 & col2.
                col1               col2
0              apple                   
1             banana         bandana123
2             carrot            care123
3        cat dog cat        cat dog dog
4           elephant           eagl232e
5            testing         testloling
6  testing1234567789            testing
7    testing testing  test testing test


In [20]:
def get_word_similarity(df, col1, col2):
    for i, row in df.iterrows():
        seq = difflib.SequenceMatcher(None, row[col1].split(), row[col2].split())
        ratio = seq.ratio()
        if ratio >= 0.1:
            print(f"Row {i+1}: {col1} and {col2} have a word similarity of {ratio*100:.2f}%.")

In [21]:
get_word_similarity(df, 'col1', 'col2')

Row 4: col1 and col2 have a word similarity of 66.67%.
Row 8: col1 and col2 have a word similarity of 40.00%.


In [34]:
data = pd.DataFrame({'col1': ['apple', 'banana', 'carrot', 'cat dog cat', 'elephant','testing', 'testing1234567789', 'testing testing'],
                   'col2': ['apple', 'bandana123', 'care123', 'cat dog dog', 'eagl232e', 'testloling', 'testing', 'test testing test']})

In [35]:
def preprocess_text(text):
    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Convert text to lowercase
    text = text.lower()
    return text

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def get_similarity_ml(data, col1, col2):
    vectorizer = TfidfVectorizer(preprocessor=preprocess_text)
    X = vectorizer.fit_transform(data[col1])
    Y = vectorizer.transform(data[col2])
    sim = cosine_similarity(X, Y)

    for i in range(sim.shape[0]):
        print(f"Row {i+1}: have a similarity of {sim[i][0]*100:.2f}% on '{data[col1][i]}' & '{data[col2][i]}'.")

In [38]:
get_similarity_ml(data, 'col1', 'col2')

Row 1: have a similarity of 100.00% on 'apple' & 'apple'.
Row 2: have a similarity of 0.00% on 'banana' & 'bandana123'.
Row 3: have a similarity of 0.00% on 'carrot' & 'care123'.
Row 4: have a similarity of 0.00% on 'cat dog cat' & 'cat dog dog'.
Row 5: have a similarity of 0.00% on 'elephant' & 'eagl232e'.
Row 6: have a similarity of 0.00% on 'testing' & 'testloling'.
Row 7: have a similarity of 0.00% on 'testing1234567789' & 'testing'.
Row 8: have a similarity of 0.00% on 'testing testing' & 'test testing test'.


In [40]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))

def preprocess(text):
    # convert to lowercase
    text = text.lower()
    # tokenize
    tokens = word_tokenize(text)
    # remove stop words and punctuation
    filtered_tokens = [token for token in tokens if token not in stop_words and token.isalpha()]
    # join back to string
    processed_text = ' '.join(filtered_tokens)
    return processed_text

def get_similarity(col1, col2):
    # preprocess the strings
    col1_processed = preprocess(col1)
    col2_processed = preprocess(col2)
    # get the set of unique words in each column
    col1_words = set(col1_processed.split())
    col2_words = set(col2_processed.split())
    # calculate Jaccard similarity
    similarity = len(col1_words.intersection(col2_words)) / len(col1_words.union(col2_words))
    return similarity

data = pd.DataFrame({'col1': ['apple', 'banana', 'carrot', 'cat dog cat', 'elephant','testing', 'testing1234567789', 'testing testing'],
                   'col2': ['apple', 'bandana123', 'care123', 'cat dog dog', 'eagl232e', 'testloling', 'testing', 'test testing test']})

for i, row in data.iterrows():
    similarity = get_similarity(row['col1'], row['col2'])
    if similarity > 0:
        print(f"Row {i+1}: have a similarity of {similarity*100:.2f}% on '{row['col1']}' & '{row['col2']}'.")
    else:
        print(f"Row {i+1}: have no similarity on '{row['col1']}' & '{row['col2']}'.")


[nltk_data] Downloading package stopwords to /Users/Andy/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /Users/Andy/nltk_data...


Row 1: have a similarity of 100.00% on 'apple' & 'apple'.
Row 2: have no similarity on 'banana' & 'bandana123'.
Row 3: have no similarity on 'carrot' & 'care123'.
Row 4: have a similarity of 100.00% on 'cat dog cat' & 'cat dog dog'.
Row 5: have no similarity on 'elephant' & 'eagl232e'.
Row 6: have no similarity on 'testing' & 'testloling'.
Row 7: have no similarity on 'testing1234567789' & 'testing'.
Row 8: have a similarity of 50.00% on 'testing testing' & 'test testing test'.


[nltk_data]   Unzipping tokenizers/punkt.zip.


In [44]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import pairwise_distances
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())

    # Remove stop words and punctuation
    stop_words = set(stopwords.words('english'))
    stop_words.update(string.punctuation)
    filtered_tokens = [token for token in tokens if token not in stop_words]

    # Return the filtered tokens as a string
    return ' '.join(filtered_tokens)

def get_similarity_ml(data, col1, col2):
    vectorizer = CountVectorizer(preprocessor=preprocess_text)
    X = vectorizer.fit_transform(data[col1])
    Y = vectorizer.transform(data[col2])

    # Convert the sparse matrices to dense matrices
    X_dense = X.toarray()
    Y_dense = Y.toarray()

    sim = 1 - pairwise_distances(X_dense, Y_dense, metric='jaccard')

    for i in range(sim.shape[0]):
        print(f"Row {i+1}: have a similarity of {sim[i][0]*100:.2f}% on '{data[col1][i]}' & '{data[col2][i]}'.")


In [45]:
get_similarity_ml(data, 'col1', 'col2')

Row 1: have a similarity of 100.00% on 'apple' & 'apple'.
Row 2: have a similarity of 0.00% on 'banana' & 'bandana123'.
Row 3: have a similarity of 0.00% on 'carrot' & 'care123'.
Row 4: have a similarity of 0.00% on 'cat dog cat' & 'cat dog dog'.
Row 5: have a similarity of 0.00% on 'elephant' & 'eagl232e'.
Row 6: have a similarity of 0.00% on 'testing' & 'testloling'.
Row 7: have a similarity of 0.00% on 'testing1234567789' & 'testing'.
Row 8: have a similarity of 0.00% on 'testing testing' & 'test testing test'.


/Users/Andy/Desktop/checking/venv/lib/python3.9/site-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
